# 랭체인 표현 언어 - LCEL (LangChain Expression Language) 심화

이전 Notebook에서 `|` 연산자를 사용해 보았습니다. 이것이 바로 **LCEL**입니다.
여기서는 LCEL을 구성하는 핵심 **Runnable** 객체들과 병렬 처리, 데이터 전달 기법 등을 깊이 있게 다룹니다.

**학습 목표:**
1. **RunnableLambda:** 사용자 정의 함수(Python Function)를 체인에 연결하기
2. **Batch & Stream:** 대량 데이터 처리와 실시간 스트리밍 출력 구현하기
3. **RunnableParallel:** 여러 작업을 동시에 병렬로 처리하여 속도와 효율성 높이기
4. **RunnablePassthrough:** 데이터를 변형 없이 전달하거나, 입력 데이터를 동적으로 추가(`assign`)하기

### 1. 환경 설정 (Environment Setup)

In [ ]:
%pip install langchain langchain-openai -Uqqq

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")

# LangSmith 설정
os.environ["LANGSMITH_TRACING"] = 'true'
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")

---### 2. RunnableLambda (사용자 정의 함수)

표준 LangChain 컴포넌트가 아닌, **내가 만든 Python 함수**를 체인 중간에 끼워 넣고 싶을 때 사용합니다.
`@chain` 데코레이터나 `RunnableLambda` 클래스로 감싸서 사용합니다.

In [ ]:
from langchain_core.runnables import RunnableLambda

# 1. 간단한 Python 함수 정의
def length_function(text):
    return len(text)

def multiple_length_function(length):
    return length * 2

# 2. Runnable로 변환 후 연결
chain = RunnableLambda(length_function) | RunnableLambda(multiple_length_function)

# 3. 실행
input_text = "LangChain is fun"
print(f"입력: '{input_text}'")
print(f"결과: {chain.invoke(input_text)}")  # 길이(16) * 2 = 32

---### 3. Batch & Stream (배치와 스트리밍)

- **batch:** 여러 개의 입력을 한 번에 처리하여 리스트로 반환 (속도 향상)
- **stream:** LLM의 응답을 한 글자씩 실시간으로 생성 (사용자 경험 향상)

In [ ]:
# 1. Batch 예제
# 위에서 만든 chain을 여러 입력에 대해 동시에 실행
inputs = ["Hello", "World", "Python", "LangChain"]
results = chain.batch(inputs)
print(f"Batch 결과: {results}")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
import time

# 2. Stream 예제
llm = ChatOpenAI(model="gpt-4o-mini")
prompt = PromptTemplate.from_template("{topic}에 대해 3문장으로 설명해줘.")
chain = prompt | llm | StrOutputParser()

print("Streams output:")
for chunk in chain.stream({"topic": "인공지능"}):
    print(chunk, end="", flush=True)
    # time.sleep(0.05)  # 속도 조절용 (실제 사용시는 불필요)

---### 4. RunnableParallel (병렬 처리)

하나의 입력을 받아 **여러 개의 작업을 동시에 수행**하고, 결과를 Dictionary 형태로 합쳐서 반환합니다.
**RAG(검색 증강 생성)**에서 가장 많이 쓰이는 패턴 중 하나입니다 (질문은 그대로 통과시키고, 문서는 검색해서 가져올 때).

In [ ]:
from langchain_core.runnables import RunnableParallel

# 예제: 하나의 주제로 n행시, 농담, 시를 동시에 생성

topic_prompt = PromptTemplate.from_template("주제: {topic}")

# 각각의 체인 정의
poem_chain = (
    PromptTemplate.from_template("{topic}에 대한 짧은 시를 지어줘")
    | llm 
    | StrOutputParser()
)

joke_chain = (
    PromptTemplate.from_template("{topic}과 관련된 웃긴 농담을 하나 해줘")
    | llm 
    | StrOutputParser()
)

# 병렬 실행 체인 구성
map_chain = RunnableParallel(
    poem=poem_chain,
    joke=joke_chain
)

# 실행
result_dict = map_chain.invoke({"topic": "커피"})

print("--- 시 ---")
print(result_dict['poem'])
print("\n--- 농담 ---")
print(result_dict['joke'])

---### 5. RunnablePassthrough (데이터 통과)

- `RunnablePassthrough()`: 입력을 변경 없이 그대로 다음 단계로 전달합니다.
- `RunnablePassthrough.assign(...)`: 입력 데이터(Dict)에 **새로운 Key-Value를 추가**할 때 사용합니다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# 1. 기본 사용 (입력 그대로 전달)
# RAG에서 자주 보이는 패턴: {'question': RunnablePassthrough(), 'context': retriever}

chain = RunnablePassthrough() | RunnableLambda(lambda x: f"입력값은: {x}")
print(chain.invoke("테스트"))

In [ ]:
# 2. .assign() 사용 (데이터 추가)
# 입력으로 들어온 dict에 새로운 값을 계산해서 합침

def count_chars(inputs):
    return len(inputs['topic'])

# 입력: {'topic': 'ChatGPT'}
# 출력: {'topic': 'ChatGPT', 'length': 7}
assign_chain = RunnablePassthrough.assign(
    length=count_chars  # 함수 실행 결과를 'length' 키에 할당
)

result = assign_chain.invoke({'topic': 'ChatGPT'})
print(f"확장된 데이터: {result}")

In [ ]:
# 3. 응용 예제: 입력 문장의 길이를 계산해서 프롬프트에 활용

final_prompt = PromptTemplate.from_template(
    "주제 '{topic}'은(는) 총 {length}글자입니다. 이 주제로 n행시를 지어주세요."
)

chain = (
    RunnablePassthrough.assign(length=count_chars) # 길이를 먼저 계산해서 추가
    | final_prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke({'topic': '인공지능'}))